# OpenAI File API

The Assistants' ability to use files to complete their tasks is a recent addition to OpenAI API and can be used for a number of purposes; for example, embeddings, semantic search, RAG and other tasks.

In this example, we will show how to use them to replicate a UI (the wonderful [Replit UI](https://replit.com) - respect, guys!) by taking a screenshot and then asking GPT-4 to replicate it for us in Typescript.

How hard can it be?

## Prerequisites

We will re-use some of the functions we defined in an [earlier Jupyter Notebook](https://codetrips.com/2024/05/03/using-openais-assistants-api/) using this neat trick (ignore the error message, what we really care is having access to the functions' definitions):

In [1]:
%run ./OpenAI\ API\ Examples.ipynb

WARN: Assistant Go Dev already exists, updating instructions.
We failed! Status: queued, None


BadRequestError: Error code: 400 - {'error': {'message': "Can't add messages to thread_BRu77Qp14lfaESbP2RMmOlHh while a run run_hscR6YM65JM3oNdTIynzCzpr is active.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

BadRequestError: Error code: 400 - {'error': {'message': "Can't add messages to thread_BRu77Qp14lfaESbP2RMmOlHh while a run run_hscR6YM65JM3oNdTIynzCzpr is active.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

Of course, you could also just copy the functions in a Python file, and just import that one: YMMV.

# Uploading a file

The [API reference](https://platform.openai.com/docs/api-reference/files) is fairly straightforward, the only gotcha may be the `file` argument in the request, which expects the actual *file* object, not the *filename*.

Other than that, uploading a file (and obtaining a reference to the underlying API object) is pretty trivial:

In [3]:
ui_image = open('./replit-ui.png', 'rb')

ui_file = client.files.create(
    file=ui_image,
    purpose="assistants",
)
print(f"Uploaded {ui_file.filename} ({ui_file.bytes} bytes) #{ui_file.id}")

Uploaded replit-ui.png (232003 bytes) #file-6KY2NfiVowf3JIV5PBRBXfsZ


In [5]:
name = "FrontEnd Dev"

instructions = """You are a dedicated frontend developer,
and have detailed knowledge of Typescript and React, and
will use your knowledge to craft well-designed Typescript
code, to implement UIs as requested by the human.

The UI components must be written using React, and the code
must be provided in separate files, with a good logical structure.

You will provide your answers in modern Typescript 5,
with appropriate comments and documentation.
The code should be complete and ready to be compiled: do not
provide just fragments, but the full code for functions and types.

Each file in your response will be delimited by three back-ticks (```)
as in Markdown, adding the language, and the path relative to the
repository root:

```typescript app/ui/components/MyComponent.ts

// Some well formatted Typescript code goes here
const foo = () => {
    console.log("Hello, Robot!");
}
```
"""

content = """Start creating the base project structure
and main files to implement a UI similar to the one
in the provided `replit-UI.png` screenshot.

Do not worry about the details and the contents of the
various panes, just organize the main window, the panes
and the top-level components that will make up the UI.
"""

# 0. We need an assistant
new_assistant(name, instructions)
ts_dev = get_asst_by_name(name=name)

#1. Add the uploaded file to its "resources"
client.beta.assistants.update(ts_dev.id,
                              tool_resources={"code_interpreter":{"file_ids":[ui_file.id]}},
                             )
ts_dev = get_asst_by_name(name)
print(f"Assistant {ts_dev.name} is now ready, with files {ts_dev.tool_resources.code_interpreter.file_ids}")

WARN: Assistant FrontEnd Dev already exists, updating instructions.
Assistant FrontEnd Dev is now ready, with files ['file-6KY2NfiVowf3JIV5PBRBXfsZ']


Note how the `file_id` is the same as the one returned earlier by the upload API call.

In [12]:
# 2. Get a Thread, and append a message to it
#thread = new_thread()
#add_msg_to_thread(thread, content)

# 4. Get a new Run, and associate it with our Thread
#    We will use the Assistant we just created.
run = new_run(thread=thread, asst_name=name)

def wait_on_run(run, thread, timeout: int = 120, interval: float = 2) -> bool:
    count = 0
    retries = timeout / interval
    while (run.status == "queued" or 
           run.status == "in_progress") and not run.status == "failed":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )    
        count += 1
        if count > retries:
            # We have exceeded the timeout, we need to first 
            # cancel the run to avoid incurring further costs.
            client.beta.threads.runs.cancel(
                thread_id=thread.id,
                run_id=run.id,
            )
        time.sleep(interval)
    return run.status == "completed"

# 5. We then ask GPT for advice
if wait_on_run(run, thread, timeout=360):
    response = get_response(thread)
    print(f"{name} says:\n{response}")
else:
    print(f"We failed! Status: {run.status} - details: {run.incomplete_details}") 

FrontEnd Dev says:
To organize the project structure and setup base components for a UI similar to the provided image, I will setup the main window, panes, and top-level components. This will include creating a file structure, a base React setup, and components representing different areas of the UI.

Here is an outline of the main GUI areas based on the screenshot:
- **Header**: A top bar possibly containing title, menu, or controls.
- **Sidebar**: A vertical panel on the left for navigation or tool options.
- **Main Content Area**: A large central panel for the primary interactive or display content.
- **Bottom Bar**: A horizontal bar at the bottom, which could serve for showing status or additional controls.

Let's create the project structure and some initial files:

### 1. Project Folder Structure
```
app/
├── ui/
│   ├── components/
│   │   ├── Header.tsx
│   │   ├── Sidebar.tsx
│   │   ├── MainContent.tsx
│   │   ├── BottomBar.tsx
│   │   └── App.tsx
│   ├── styles/
│   │   └── 

Et voila! A Replit clone is served for you by your friendly Code Assistant.